In [ ]:
!pip install generative-ai-hub-sdk[all] --break-system-packages
!pip install ipywidgets --break-system-packages
!pip install pandas --break-system-packages

# kernel restart required!!!


In [11]:
!pip show generative-ai-hub-sdk

Name: generative-ai-hub-sdk
Version: 3.8.0
Summary: generative AI hub SDK
Home-page: https://www.sap.com/
Author: SAP SE
Author-email: 
License: SAP DEVELOPER LICENSE AGREEMENT
Location: /Users/I064538/Documents/SAP/dev/sap-genai-hub-with-sap-hana-cloud-vector-engine/genai_hub_env/lib/python3.12/site-packages
Requires: ai-core-sdk, click, dacite, openai, overloading, packaging, pydantic
Required-by: 


In [12]:
# Test embeddings

from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="SAP Generative AI Hub is awesome!",
    model_name="text-embedding-ada-002"
    
)
print(response.data)

[Embedding(embedding=[-0.001376426313072443, -0.022090168669819832, -0.005044370889663696, -0.016446400433778763, -0.03345852345228195, 0.014897393994033337, -0.008075034245848656, -0.014250853098928928, 0.0055023375898599625, -0.05840425193309784, 0.01485698577016592, -0.0007850260008126497, 0.01038507279008627, -0.02420489862561226, -0.0006524344789795578, 0.005734688602387905, 0.027828224003314972, -0.004690793342888355, 0.009731796570122242, -0.012425719760358334, -0.021106885746121407, 0.027464544400572777, 0.007381348870694637, -0.019557880237698555, -0.02378734014928341, 0.022548135370016098, 0.020366057753562927, -0.022898346185684204, -0.00685603404417634, -0.020891372114419937, 0.0011895353673025966, 0.006590009201318026, -0.021942002698779106, -0.005320497788488865, -0.001679492648690939, 0.01465494092553854, -0.0078123765997588634, -0.005374376196414232, -0.005724586546421051, 0.0041621108539402485, 0.019625229761004448, 0.0029229065403342247, -0.0027882102876901627, -0.028

In [13]:
# YOUR_API_URL="https://api.ai.prodeuonly.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/de066cc850630a68"
YOUR_API_URL = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d26ce2f31ba6ff8a"

In [14]:
import json
# import pandas as pd
from gen_ai_hub.orchestration.utils import load_text_file
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, ProfileEntity
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter
from gen_ai_hub.orchestration.models.config import OrchestrationConfig, Template
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.service import OrchestrationService

In [ ]:
"""PERSON: Represents personal names.
ORG: Represents organizational names.
UNIVERSITY: Represents educational institutions.
LOCATION: Represents geographical locations.
EMAIL: Represents email addresses.
PHONE: Represents phone numbers.
ADDRESS: Represents physical addresses.
SAP_IDS_INTERNAL: Represents internal SAP identifiers.
SAP_IDS_PUBLIC: Represents public SAP identifiers.
URL: Represents URLs.
USERNAME_PASSWORD: Represents usernames and passwords.
NATIONAL_ID: Represents national identification numbers.
IBAN: Represents International Bank Account Numbers.
SSN: Represents Social Security Numbers.
CREDIT_CARD_NUMBER: Represents credit card numbers.
PASSPORT: Represents passport numbers.
DRIVING_LICENSE: Represents driving license numbers.
NATIONALITY: Represents nationality information.
RELIGIOUS_GROUP: Represents religious group affiliation.
POLITICAL_GROUP: Represents political group affiliation.
PRONOUNS_GENDER: Represents pronouns and gender identity.
GENDER: Represents gender information.
SEXUAL_ORIENTATION: Represents sexual orientation.
TRADE_UNION: Represents trade union membership.
SENSITIVE_DATA: Represents any other sensitive information."""

In [15]:
# Load the JSON configuration file
file_path = 'data/ModelOrchConfig_1.json'
def load_json_config(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Extract information from the JSON config and generate the Python configuration
def update_python_config(json_config):
    # Extract relevant information from the JSON
    llm_model_name = json_config['module_configurations']['llm_module_config']['model_name']
    input_filter_config = json_config['module_configurations']['filtering_module_config']['input']['filters'][0]['config']
    output_filter_config = json_config['module_configurations']['filtering_module_config']['output']['filters'][0]['config']
    masking_entities = json_config['module_configurations']['masking_module_config']['masking_providers'][0]['entities']
    system_message_content = json_config['module_configurations']['templating_module_config']['template'][0]['content']
    user_message_content = json_config['module_configurations']['templating_module_config']['template'][1]['content']
    # Generate the input and output filter objects based on the extracted configuration
    # system_message_content = "greet the user {{?user}} \nyou are an python agent give a code for the scenario {{?code}} "

    input_filter = AzureContentFilter(
        hate=input_filter_config['Hate'],
        sexual=input_filter_config['Sexual'],
        self_harm=input_filter_config['SelfHarm'],
        violence=input_filter_config['Violence']
    )
    output_filter = AzureContentFilter(
        hate=output_filter_config['Hate'],
        sexual=output_filter_config['Sexual'],
        self_harm=output_filter_config['SelfHarm'],
        violence=output_filter_config['Violence']
    )
    # Generate the data masking providers based on the extracted entities
    masking_providers = []
    for entity in masking_entities:
        entity_type = entity['type']
        if entity_type == 'profile-email':
            masking_providers.append(ProfileEntity.EMAIL)
        elif entity_type == 'profile-phone':
            masking_providers.append(ProfileEntity.PHONE)
        elif entity_type == 'profile-gender':
            masking_providers.append(ProfileEntity.GENDER)
        elif entity_type == 'profile-location':
            masking_providers.append(ProfileEntity.LOCATION)
        elif entity_type == 'profile-nationalid':
            masking_providers.append(ProfileEntity.NATIONAL_ID)
        elif entity_type == 'profile-nationality':
            masking_providers.append(ProfileEntity.NATIONALITY)
        elif entity_type == 'profile-org':
            masking_providers.append(ProfileEntity.ORG)
        elif entity_type == 'profile-person':
            masking_providers.append(ProfileEntity.PERSON)
        elif entity_type == 'profile-university':
            masking_providers.append(ProfileEntity.UNIVERSITY)
        elif entity_type == 'profile-url':
            masking_providers.append(ProfileEntity.URL)
        elif entity_type == 'profile-username-password':
            masking_providers.append(ProfileEntity.USERNAME_PASSWORD)

    # Generate the data masking config
    data_masking = DataMasking(
        providers=[SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,
            entities=masking_providers
        )]
    )

    # Return the updated Python config
    return OrchestrationConfig(
        template=Template(
            messages=[
                SystemMessage(system_message_content),
                UserMessage(user_message_content),
            ]),
        llm=LLM(name=llm_model_name),
        data_masking=data_masking,
        input_filters=[input_filter],
        output_filters=[output_filter]
    )

In [16]:
# Assuming you have a JSON file path 'ModelOrchConfig_1.json'
json_file_path = 'data/ModelOrchConfig_1.json'
json_config = load_json_config(json_file_path)

# Update the configuration
config_ = update_python_config(json_config)

orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config_)
# orchestration_service = OrchestrationService(api_url=YOUR_API_URL)

# Execute Orchestration Service
result = orchestration_service.run(
    config=config_,
    template_values=[
        TemplateValue(name="candidate_resume", value="John Doe \n1234 Data St, San Francisco, CA 94101 \n(123) 456-7890 \njohndoe@email.com \nLinkedIn Profile \nGitHub Profile \nObjective \nDetail-oriented Data Scientist with 3+ years of experience in data analysis, statistical modeling, and machine learning. Seeking to leverage expertise in predictive modeling and data visualization to help drive data-informed decision-making at [Company Name]. \nEducation \nMaster of Science in Data Science \nUniversity of California, Berkeley \nGraduated: May 2021 \nBachelor of Science in Computer Science \nUniversity of California, Los Angeles \nGraduated: May 2019 \nTechnical Skills \nProgramming Languages: Python, R, SQL, Java \nData Analysis & Visualization: Pandas, NumPy, Matplotlib, Seaborn, Tableau \nMachine Learning: Scikit-learn, TensorFlow, Keras, XGBoost \nBig Data Technologies: Hadoop, Spark \nDatabases: MySQL, PostgreSQL \nVersion Control: Git \nProfessional Experience  \nData Scientist \nDataCorp Inc., San Francisco, CA \nJune 2021 – Present \nDeveloped predictive models to optimize marketing campaigns, which increased ROI by 20%. \nConducted in-depth data analysis using Python and SQL to identify trends and patterns in large datasets. \nCollaborated with cross-functional teams to implement data-driven strategies that improved customer satisfaction scores by 15%. \nCreated interactive dashboards using Tableau to visualize KPIs for stakeholders.  \nData Analyst Intern \nAnalytics Solutions, Los Angeles, CA \nJune 2020 – August 2020 \nAnalyzed large datasets to identify opportunities for business growth and improvement.\nAssisted in the development of automated reporting tools using Python and Excel. \nWorked with data visualization tools to create insightful reports for management. \nProjects \nCustomer Segmentation Analysis \nConducted K-means clustering on customer data to segment the customer base into distinct groups, enabling targeted marketing strategies. \nPredictive Stock Price Modeling \nBuilt a predictive model using time series analysis to forecast stock prices, achieving an accuracy rate of 85%. \nSentiment Analysis on Social Media \nImplemented natural language processing techniques to analyze sentiment from tweets, providing insights into public opinion on various topics. \nCertifications \nCertified Data Scientist (CDS) – Data Science Council of America \nMachine Learning Specialization – Coursera by Stanford University \nProfessional Affiliations \nMember, Association for Computing Machinery (ACM) \nMember, Data Science Society \nReferences \nAvailable upon request. \nPersonal Interests \n- I absolutely love exploring new technologies and working on innovative projects. \n- I enjoy reading books, especially on artificial intelligence and machine learning. \n- I hate people who are dishonest and unreliable. \n- I love traveling and experiencing new cultures. \n- I enjoy playing video games, especially competitive ones. \n- I hate being stuck in a routine; I always seek new challenges and growth opportunities. \n- I hate working in Azure cloud -\"Azure cloud is the most irritating platform i have ever used\" \n\n")
    ]
)

# Check for masked entities in the response
masked_result = result.orchestration_result.choices[0].message.content
print(masked_result)

The candidate is a Data Scientist with over 3 years of experience in data analysis, statistical modeling, and machine learning. They hold a Master of Science in Data Science and a Bachelor of Science in Computer Science, with educational achievements completed by May 2021 and May 2019, respectively. Their technical skills include proficiency in Python, R, SQL, Java, and big data technologies like Hadoop and Spark. Professionally, they have worked at MASKED_ORG since June 2021, where they've developed predictive models that increased marketing ROI by 20% and improved customer satisfaction by 15%. Previously, they interned at Analytics Solutions, where they analyzed large datasets and assisted in creating automated reporting tools. Key projects include customer segmentation analysis using K-means clustering and predictive stock price modeling with an 85% accuracy rate. They also possess certifications like Certified Data Scientist and Machine Learning Specialization. The candidate is a m